### Vetores e Espaço Vetorial em Sistemas de Recomendação

Instalando e carregando pacotes

In [1]:
# !pip install nltk
# !pip install scikit-learn

In [2]:
import ast
import nltk
import sklearn
import numpy as np
import pandas as pd
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

pd.options.mode.chained_assignment = None

Carregando e Compreendendo os dados

In [3]:
# dataset de filmes
df_movies = pd.read_csv('data/dataset_filmes.csv')

In [4]:
df_movies.shape

(4803, 20)

In [5]:
# Amostra de dados
df_movies.sample(3)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
1930,0,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",NaN,21338,"[{""id"": 1568, ""name"": ""undercover""}, {""id"": 18...",en,Stone Cold,"Joe Huff (Brian Bosworth) is a tough, loner co...",3.623002,"[{""name"": ""Stone Group Pictures"", ""id"": 3470},...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",1991-05-17,0,92.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,A cop who enforces his own brand of justice.,Stone Cold,5.7,34
1363,35000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 35, ""nam...",NaN,10054,"[{""id"": 848, ""name"": ""double life""}, {""id"": 97...",en,Spy Kids,Carmen and Juni think their parents are boring...,21.543389,"[{""name"": ""Dimension Films"", ""id"": 7405}, {""na...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2001-03-18,147934180,88.0,"[{""iso_639_1"": ""es"", ""name"": ""Espa\u00f1ol""}, ...",Released,Real spies...only smaller,Spy Kids,5.5,867
4363,1000000,"[{""id"": 99, ""name"": ""Documentary""}, {""id"": 104...",NaN,159014,[],en,20 Feet from Stardom,Backup singers live in a world that lies just ...,5.247956,"[{""name"": ""Tremolo Productions"", ""id"": 58637}]","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2013-06-14,0,89.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,NaN,20 Feet from Stardom,7.4,85


In [6]:
# Dataset de elenco dos filmes
df_actors = pd.read_csv('data/dataset_elenco.csv')

In [7]:
df_actors.shape

(4803, 4)

In [8]:
df_actors.head()

,movie_id,title,cast,crew
0,19995,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,285,Pirates of the Caribbean: At World's End,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,206647,Spectre,"[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."
3,49026,The Dark Knight Rises,"[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de..."
4,49529,John Carter,"[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de..."


Organização dos dados de texto

Existe uma relaçao entre os 2 datasets. Vamos fazer o merge pela coluna comum entre eles, a coluna title.

In [9]:
df_movies.columns

Index(['budget', 'genres', 'homepage', 'id', 'keywords', 'original_language',
       'original_title', 'overview', 'popularity', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'vote_average',
       'vote_count'],
      dtype='object')

In [10]:
df_actors.columns

Index(['movie_id', 'title', 'cast', 'crew'], dtype='object')

In [11]:
# Merge os dataframes
df_movies = df_movies.merge(df_actors, on = 'title')

In [12]:
df_movies.shape

(4809, 23)

In [13]:
df_movies.sample()

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,movie_id,cast,crew
3475,0,"[{""id"": 18, ""name"": ""Drama""}, {""id"": 10749, ""n...",NaN,12509,"[{""id"": 2679, ""name"": ""artist""}, {""id"": 3836, ...",en,Pollock,"In August of 1949, Life Magazine ran a banner ...",8.483531,"[{""name"": ""Fred Berner Films"", ""id"": 89312}, {...",...,122.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,A true portrait of life and art.,Pollock,6.6,91,12509,"[{""cast_id"": 1, ""character"": ""Jackson Pollock""...","[{""credit_id"": ""52fe44f19251416c75046447"", ""de..."


In [14]:
df_actors[df_actors['movie_id'] == 313922]

,movie_id,title,cast,crew
3640,313922,Green Room,"[{""cast_id"": 1, ""character"": ""Pat"", ""credit_id...","[{""credit_id"": ""572dfbc5c3a3680f3f00288a"", ""de..."


In [15]:
df_movies.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4809 entries, 0 to 4808
Data columns (total 23 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   budget                4809 non-null   int64  
 1   genres                4809 non-null   object 
 2   homepage              1713 non-null   object 
 3   id                    4809 non-null   int64  
 4   keywords              4809 non-null   object 
 5   original_language     4809 non-null   object 
 6   original_title        4809 non-null   object 
 7   overview              4806 non-null   object 
 8   popularity            4809 non-null   float64
 9   production_companies  4809 non-null   object 
 10  production_countries  4809 non-null   object 
 11  release_date          4808 non-null   object 
 12  revenue               4809 non-null   int64  
 13  runtime               4807 non-null   float64
 14  spoken_languages      4809 non-null   object 
 15  status               

Muitas colunas parecem não ser relevantes para nossa análise. Vamos manter apenas o que é relevantes.

In [16]:
# Filtrando o dataframe e mantendo somente as colunas consideradas mais relevantes
df_movies = df_movies[['movie_id', 'title', 'overview', 'genres', 'keywords', 'cast', 'crew']]

A decisão acima é sua, Cientista de Dados, com base no seu conhecimento sobre a área de negócio ou sua percepção sobre os dados. Tome a decisão, justifique-a e siga em frente no processo de análise. Se mais tarde achou que não foi a melhor decisão, então volte, mude a decisão, justifique novamente e siga em frente até chegar ao melhor resultado possível.

In [17]:
df_movies.head(10)

,movie_id,title,overview,genres,keywords,cast,crew
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di...","[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...","[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,285,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...","[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...","[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...","[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,206647,Spectre,A cryptic message from Bond’s past sends him o...,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...","[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."
3,49026,The Dark Knight Rises,Following the death of District Attorney Harve...,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...","[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...","[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de..."
4,49529,John Carter,"John Carter is a war-weary, former military ca...","[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 818, ""name"": ""based on novel""}, {""id"":...","[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de..."
5,559,Spider-Man 3,The seemingly invincible Spider-Man goes up ag...,"[{""id"": 14, ""name"": ""Fantasy""}, {""id"": 28, ""na...","[{""id"": 851, ""name"": ""dual identity""}, {""id"": ...","[{""cast_id"": 30, ""character"": ""Peter Parker / ...","[{""credit_id"": ""52fe4252c3a36847f80151a5"", ""de..."
6,38757,Tangled,When the kingdom's most wanted-and most charmi...,"[{""id"": 16, ""name"": ""Animation""}, {""id"": 10751...","[{""id"": 1562, ""name"": ""hostage""}, {""id"": 2343,...","[{""cast_id"": 34, ""character"": ""Flynn Rider (vo...","[{""credit_id"": ""52fe46db9251416c91062101"", ""de..."
7,99861,Avengers: Age of Ultron,When Tony Stark tries to jumpstart a dormant p...,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 8828, ""name"": ""marvel comic""}, {""id"": ...","[{""cast_id"": 76, ""character"": ""Tony Stark / Ir...","[{""credit_id"": ""55d5f7d4c3a3683e7e0016eb"", ""de..."
8,767,Harry Potter and the Half-Blood Prince,"As Harry begins his sixth year at Hogwarts, he...","[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...","[{""id"": 616, ""name"": ""witch""}, {""id"": 2343, ""n...","[{""cast_id"": 3, ""character"": ""Harry Potter"", ""...","[{""credit_id"": ""52fe4273c3a36847f801fab1"", ""de..."
9,209112,Batman v Superman: Dawn of Justice,Fearing the actions of a god-like Super Hero l...,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 849, ""name"": ""dc comics""}, {""id"": 7002...","[{""cast_id"": 18, ""character"": ""Bruce Wayne / B...","[{""credit_id"": ""553bf23692514135c8002886"", ""de..."


In [18]:
df_movies.shape

(4809, 7)

### Análise Exploratória d Limpeza dos Dados

In [19]:
df_movies.shape

(4809, 7)

In [20]:
# checando os valores ausentes
df_movies.isnull().sum()

movie_id    0
title       0
overview    3
genres      0
keywords    0
cast        0
crew        0
dtype: int64

In [21]:
# removendo as linhas com valores ausentes
df_movies.dropna(inplace = True)

In [22]:
df_movies.isnull().sum()

movie_id    0
title       0
overview    0
genres      0
keywords    0
cast        0
crew        0
dtype: int64

In [23]:
# checando se há valores duplicados
df_movies.duplicated().sum()

np.int64(0)

In [24]:
df_movies.shape

(4806, 7)

### Processamento de Text com Abstract Syntax Tree

https://docs.python.org/3/library/ast.html

Vamos começar ajustando a coluna de gênero de filme. Nesta coluna precisamos apenas dos nomes e não dos ids.

In [25]:
df_movies.head()

,movie_id,title,overview,genres,keywords,cast,crew
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di...","[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...","[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,285,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...","[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...","[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...","[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,206647,Spectre,A cryptic message from Bond’s past sends him o...,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...","[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."
3,49026,The Dark Knight Rises,Following the death of District Attorney Harve...,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...","[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...","[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de..."
4,49529,John Carter,"John Carter is a war-weary, former military ca...","[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 818, ""name"": ""based on novel""}, {""id"":...","[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de..."


O módulo 'ast' ajuda a descobrir programaticamente como é a gramática atual de um objeto, processando árvores da grmática de sintaxe abstrata.

In [26]:
# Exemplo
ast.literal_eval('[{"name": "Matheus", "age": 30}, {"name": "Noah", "age": 3}, {"name": "Cleubiani", "age": 28}]')

[{'name': 'Matheus', 'age': 30},
 {'name': 'Noah', 'age': 3},
 {'name': 'Cleubiani', 'age': 28}]

In [27]:
def convert_obj(obj):
    data = []

    for i in ast.literal_eval(obj):
        data.append(i['name'])

    return data

In [28]:
test_converter = convert_obj('[{"name": "Matheus", "age": 30}, {"name": "Noah", "age": 3}, {"name": "Cleubiani", "age": 28}]')

In [29]:
test_converter

['Matheus', 'Noah', 'Cleubiani']

In [30]:
df_movies['genres'] = df_movies['genres'].apply(convert_obj)

In [31]:
df_movies.head()

,movie_id,title,overview,genres,keywords,cast,crew
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di...","[Action, Adventure, Fantasy, Science Fiction]","[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...","[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,285,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...","[Adventure, Fantasy, Action]","[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...","[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,206647,Spectre,A cryptic message from Bond’s past sends him o...,"[Action, Adventure, Crime]","[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...","[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."
3,49026,The Dark Knight Rises,Following the death of District Attorney Harve...,"[Action, Crime, Drama, Thriller]","[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...","[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de..."
4,49529,John Carter,"John Carter is a war-weary, former military ca...","[Action, Adventure, Science Fiction]","[{""id"": 818, ""name"": ""based on novel""}, {""id"":...","[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de..."


In [32]:
df_movies['keywords'] = df_movies['keywords'].apply(convert_obj)

In [33]:
df_movies.head()

,movie_id,title,overview,genres,keywords,cast,crew
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di...","[Action, Adventure, Fantasy, Science Fiction]","[culture clash, future, space war, space colon...","[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,285,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...","[Adventure, Fantasy, Action]","[ocean, drug abuse, exotic island, east india ...","[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,206647,Spectre,A cryptic message from Bond’s past sends him o...,"[Action, Adventure, Crime]","[spy, based on novel, secret agent, sequel, mi...","[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."
3,49026,The Dark Knight Rises,Following the death of District Attorney Harve...,"[Action, Crime, Drama, Thriller]","[dc comics, crime fighter, terrorist, secret i...","[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de..."
4,49529,John Carter,"John Carter is a war-weary, former military ca...","[Action, Adventure, Science Fiction]","[based on novel, mars, medallion, space travel...","[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de..."


Vamos aplicar o 'ast' à coluna de cast (elenco) mas extrair sobre 3 membros do elenco. Isso deve ser suficiente

In [34]:
def convert_obj_limit(obj):
    data = []

    for index, element in enumerate(ast.literal_eval(obj)):
        if index < 3:
            data.append(element['name'])
        else:
            break

    return data

In [35]:
convert_obj_limit(df_movies.iloc[2]['cast']) # test

['Daniel Craig', 'Christoph Waltz', 'Léa Seydoux']

In [36]:
df_movies['cast'] = df_movies['cast'].apply(convert_obj_limit)

In [37]:
df_movies.head(2)

,movie_id,title,overview,genres,keywords,cast,crew
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di...","[Action, Adventure, Fantasy, Science Fiction]","[culture clash, future, space war, space colon...","[Sam Worthington, Zoe Saldana, Sigourney Weaver]","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,285,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...","[Adventure, Fantasy, Action]","[ocean, drug abuse, exotic island, east india ...","[Johnny Depp, Orlando Bloom, Keira Knightley]","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."


In [38]:
def fetch_director(obj):
    director = []

    for element in ast.literal_eval(obj):
        if element['job'].lower() == 'director':
            director.append(element['name'])
            break

    return director

In [39]:
fetch_director(df_movies.iloc[2]['crew'])

['Sam Mendes']

In [40]:
df_movies['crew'] = df_movies['crew'].apply(fetch_director)

In [41]:
df_movies.head()

,movie_id,title,overview,genres,keywords,cast,crew
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di...","[Action, Adventure, Fantasy, Science Fiction]","[culture clash, future, space war, space colon...","[Sam Worthington, Zoe Saldana, Sigourney Weaver]",[James Cameron]
1,285,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...","[Adventure, Fantasy, Action]","[ocean, drug abuse, exotic island, east india ...","[Johnny Depp, Orlando Bloom, Keira Knightley]",[Gore Verbinski]
2,206647,Spectre,A cryptic message from Bond’s past sends him o...,"[Action, Adventure, Crime]","[spy, based on novel, secret agent, sequel, mi...","[Daniel Craig, Christoph Waltz, Léa Seydoux]",[Sam Mendes]
3,49026,The Dark Knight Rises,Following the death of District Attorney Harve...,"[Action, Crime, Drama, Thriller]","[dc comics, crime fighter, terrorist, secret i...","[Christian Bale, Michael Caine, Gary Oldman]",[Christopher Nolan]
4,49529,John Carter,"John Carter is a war-weary, former military ca...","[Action, Adventure, Science Fiction]","[based on novel, mars, medallion, space travel...","[Taylor Kitsch, Lynn Collins, Samantha Morton]",[Andrew Stanton]


### Limpeza de dados de texto

In [42]:
# separamos a string da coluna overview por espaço em branco
df_movies['overview'] = df_movies['overview'].apply(lambda x: x.split())

In [43]:
df_movies.head()

,movie_id,title,overview,genres,keywords,cast,crew
0,19995,Avatar,"[In, the, 22nd, century,, a, paraplegic, Marin...","[Action, Adventure, Fantasy, Science Fiction]","[culture clash, future, space war, space colon...","[Sam Worthington, Zoe Saldana, Sigourney Weaver]",[James Cameron]
1,285,Pirates of the Caribbean: At World's End,"[Captain, Barbossa,, long, believed, to, be, d...","[Adventure, Fantasy, Action]","[ocean, drug abuse, exotic island, east india ...","[Johnny Depp, Orlando Bloom, Keira Knightley]",[Gore Verbinski]
2,206647,Spectre,"[A, cryptic, message, from, Bond’s, past, send...","[Action, Adventure, Crime]","[spy, based on novel, secret agent, sequel, mi...","[Daniel Craig, Christoph Waltz, Léa Seydoux]",[Sam Mendes]
3,49026,The Dark Knight Rises,"[Following, the, death, of, District, Attorney...","[Action, Crime, Drama, Thriller]","[dc comics, crime fighter, terrorist, secret i...","[Christian Bale, Michael Caine, Gary Oldman]",[Christopher Nolan]
4,49529,John Carter,"[John, Carter, is, a, war-weary,, former, mili...","[Action, Adventure, Science Fiction]","[based on novel, mars, medallion, space travel...","[Taylor Kitsch, Lynn Collins, Samantha Morton]",[Andrew Stanton]


In [44]:
# Remoção de espaço por vazio
df_movies['genres'] = df_movies['genres'].apply(lambda x: [i.replace(' ', '') for i in x])
df_movies['overview'] = df_movies['overview'].apply(lambda x: [i.replace(' ', '') for i in x])
df_movies['keywords'] = df_movies['keywords'].apply(lambda x: [i.replace(' ', '') for i in x])
df_movies['cast'] = df_movies['cast'].apply(lambda x: [i.replace(' ', '') for i in x])
df_movies['crew'] = df_movies['crew'].apply(lambda x: [i.replace(' ', '') for i in x])

In [45]:
df_movies.head()

,movie_id,title,overview,genres,keywords,cast,crew
0,19995,Avatar,"[In, the, 22nd, century,, a, paraplegic, Marin...","[Action, Adventure, Fantasy, ScienceFiction]","[cultureclash, future, spacewar, spacecolony, ...","[SamWorthington, ZoeSaldana, SigourneyWeaver]",[JamesCameron]
1,285,Pirates of the Caribbean: At World's End,"[Captain, Barbossa,, long, believed, to, be, d...","[Adventure, Fantasy, Action]","[ocean, drugabuse, exoticisland, eastindiatrad...","[JohnnyDepp, OrlandoBloom, KeiraKnightley]",[GoreVerbinski]
2,206647,Spectre,"[A, cryptic, message, from, Bond’s, past, send...","[Action, Adventure, Crime]","[spy, basedonnovel, secretagent, sequel, mi6, ...","[DanielCraig, ChristophWaltz, LéaSeydoux]",[SamMendes]
3,49026,The Dark Knight Rises,"[Following, the, death, of, District, Attorney...","[Action, Crime, Drama, Thriller]","[dccomics, crimefighter, terrorist, secretiden...","[ChristianBale, MichaelCaine, GaryOldman]",[ChristopherNolan]
4,49529,John Carter,"[John, Carter, is, a, war-weary,, former, mili...","[Action, Adventure, ScienceFiction]","[basedonnovel, mars, medallion, spacetravel, p...","[TaylorKitsch, LynnCollins, SamanthaMorton]",[AndrewStanton]


### Preparando os Dados para vetorização em outro espaço vetorial

In [46]:
# Criamos a coluna de tags, nesse caso um vetor de strings com os valores das colunas
df_movies['tags'] = df_movies['overview'] + df_movies['genres'] + df_movies['keywords'] + df_movies['cast'] + df_movies['crew']

In [47]:
df_movies.sample()

,movie_id,title,overview,genres,keywords,cast,crew,tags
1645,10658,Howard the Duck,"[A, scientific, experiment, unknowingly, bring...","[Comedy, Fantasy, ScienceFiction]","[duck, physicist, extraterrestrial, anthropomo...","[LeaThompson, JeffreyJones, TimRobbins]",[WillardHuyck],"[A, scientific, experiment, unknowingly, bring..."


In [48]:
df_movies[df_movies['movie_id'] == 242582]['tags']

3237    [When, Lou, Bloom,, desperate, for, work,, mus...
Name: tags, dtype: object

In [49]:
# list(filter(lambda x: x != "" and x != None, df_movies[df_movies['movie_id'] == 242582]['tags']))

In [50]:
# dataset final
df_movies = df_movies[['movie_id', 'title', 'tags']]

In [51]:
df_movies.head()

,movie_id,title,tags
0,19995,Avatar,"[In, the, 22nd, century,, a, paraplegic, Marin..."
1,285,Pirates of the Caribbean: At World's End,"[Captain, Barbossa,, long, believed, to, be, d..."
2,206647,Spectre,"[A, cryptic, message, from, Bond’s, past, send..."
3,49026,The Dark Knight Rises,"[Following, the, death, of, District, Attorney..."
4,49529,John Carter,"[John, Carter, is, a, war-weary,, former, mili..."


In [52]:
# join das strings para simplificar o vetor
df_movies['tags'] = df_movies['tags'].apply(lambda x: " ".join(x))

In [53]:
df_movies.head()

,movie_id,title,tags
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di..."
1,285,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha..."
2,206647,Spectre,A cryptic message from Bond’s past sends him o...
3,49026,The Dark Knight Rises,Following the death of District Attorney Harve...
4,49529,John Carter,"John Carter is a war-weary, former military ca..."


In [54]:
df_movies['tags'] = df_movies['tags'].apply(lambda x: x.lower())

In [55]:
df_movies.sample()

,movie_id,title,tags
1809,36355,Flipper,"sandy ricks is sent by his mom to coral key, a..."


### Parse e vetorização

In [56]:
# criamos o parse
parse_ps = PorterStemmer()

Stemming é o processo de redução de uma palavra ao seu radical que está ligado a sufixos e prefixos ou às raízes de palavras conhecidas "lemmas". Stemming é importante na compreesão de linguagem natural e processamento de linguagem natural.

In [57]:
def stem(text):
    # cria uma lista vazia chamada y para armazenas as palavras após o stemming
    y = []
    for i in text.split():
        y.append(parse_ps.stem(i))

    return " ".join(y)

In [58]:
df_movies['tags'] = df_movies['tags'].apply(stem)

In [59]:
df_movies.head()

,movie_id,title,tags
0,19995,Avatar,"in the 22nd century, a parapleg marin is dispa..."
1,285,Pirates of the Caribbean: At World's End,"captain barbossa, long believ to be dead, ha c..."
2,206647,Spectre,a cryptic messag from bond’ past send him on a...
3,49026,The Dark Knight Rises,follow the death of district attorney harvey d...
4,49529,John Carter,"john carter is a war-weary, former militari ca..."


In [60]:
# cria o vetorizador com no máximo 5000 atributos
cv = CountVectorizer(max_features=5000, stop_words='english')

A vetorização no contexto de processamento de linguagem natual (NLP) é o processo de converter texto em uma representação numérica, geralmente na forma de vetores. Este processo é fundamental para que algoritmos de aprendizado de máquina possam trabalhar com dados textuais, uma vez que eles requerem entradas numéricas.

In [61]:
# criar vetores para as tags
vectors = cv.fit_transform(df_movies['tags']).toarray()

In [62]:
len(cv.get_feature_names_out())

5000

In [63]:
type(vectors)

numpy.ndarray

In [64]:
vectors

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], shape=(4806, 5000))

In [65]:
# para visualizar todas as colunas do array
np.set_printoptions(threshold=np.inf)

In [66]:
vectors[0]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 2, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

### Cálculo de distância de vetores

A célula abaixo trabalha com os vetores no espaço de dimensão para calcular a distância matemática entre eles usando a distância de cosseno.

In [67]:
# Calcula a similaridade entre os vetores calculando as distâncias entre eles
similaridades = cosine_similarity(vectors)

### Construindo o sistema de Recomendação

In [70]:
def sistema_recomendacao(movie):
    # obtém o índice do filme passado como argumento (o que o usuário assistiu)
    index = df_movies[df_movies['title'] == movie].index[0]

    # verificamos então os filmes com vetores de menor distância para o filme passado como argumento
    distances = sorted(list(enumerate(similaridades[index])), reverse=True, key=lambda x : x[1])

    # e então consideramos os 5 filmes com menor distância, ou seja, maior similaridade
    for i in distances[1:6]:
        print(df_movies.iloc[i[0]].title)

# Aplicando o sistema de Recomendação

In [71]:
sistema_recomendacao('Avengers: Age of Ultron')

Iron Man 3
Iron Man 2
Iron Man
Thor
The Avengers


In [72]:
sistema_recomendacao('Jurassic Park')

Jurassic World
The Lost World: Jurassic Park
Jurassic Park III
Sea Rex 3D: Journey to a Prehistoric World
Land of the Lost


In [73]:
sistema_recomendacao('Avatar')

Aliens vs Predator: Requiem
Aliens
Falcon Rising
Independence Day
Titan A.E.
